<a href="https://colab.research.google.com/github/loissnijders/Process-mining/blob/main/ARIMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import time
start = time.time()

In [2]:
from google.colab import drive

In [3]:
import io
import pandas as pd
import re
from datetime import timedelta
import numpy as np
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers import Activation, Dense
from sklearn.preprocessing import MinMaxScaler
import sklearn.preprocessing as preprocessing
import seaborn as sn
import matplotlib.pyplot as plt

In [4]:
!wget -O training.csv https://surfdrive.surf.nl/files/index.php/s/aNVxBfvZl273kKN/download   
!wget -O test.csv https://surfdrive.surf.nl/files/index.php/s/0lJ3j0RlMeychGj/download   


path_test = '/content/test.csv'
path_train = '/content/training.csv'

--2022-03-15 09:15:43--  https://surfdrive.surf.nl/files/index.php/s/aNVxBfvZl273kKN/download
Resolving surfdrive.surf.nl (surfdrive.surf.nl)... 145.100.27.67, 2001:610:108:203b:0:a11:da7a:5afe
Connecting to surfdrive.surf.nl (surfdrive.surf.nl)|145.100.27.67|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26049572 (25M) [text/csv]
Saving to: ‘training.csv’

training.csv        100%[===================>]  24.84M  17.9MB/s    in 1.4s    

2022-03-15 09:15:45 (17.9 MB/s) - ‘training.csv’ saved [26049572/26049572]

--2022-03-15 09:15:45--  https://surfdrive.surf.nl/files/index.php/s/0lJ3j0RlMeychGj/download
Resolving surfdrive.surf.nl (surfdrive.surf.nl)... 145.100.27.67, 2001:610:108:203b:0:a11:da7a:5afe
Connecting to surfdrive.surf.nl (surfdrive.surf.nl)|145.100.27.67|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5819692 (5.5M) [text/csv]
Saving to: ‘test.csv’

test.csv            100%[===================>]   5.55M  7.45MB/s    in 0.

In [5]:
df_train = pd.read_csv (path_train)
df_test = pd.read_csv (path_test)

Cleaning the data

In [6]:
# Keeping in test set only cases that started in test and not in train

train_cases = list(set(list(df_train['case concept:name'])))
test_cases = list(set(list(df_test['case concept:name'])))
test_keep = [item for item in test_cases if item not in train_cases]

df_test = df_test[df_test['case concept:name'].isin(test_keep)]

df_show = df_test.copy()

In [7]:
df_train

,eventID,case concept:name,case REG_DATE,case AMOUNT_REQ,event concept:name,event lifecycle:transition,event time:timestamp
0,0,173688,2011-10-01T00:38:44.546+02:00,20000,A_SUBMITTED,COMPLETE,01-10-2011 00:38:44.546
1,1,173688,2011-10-01T00:38:44.546+02:00,20000,A_PARTLYSUBMITTED,COMPLETE,01-10-2011 00:38:44.880
2,2,173688,2011-10-01T00:38:44.546+02:00,20000,A_PREACCEPTED,COMPLETE,01-10-2011 00:39:37.906
3,3,173688,2011-10-01T00:38:44.546+02:00,20000,W_Completeren aanvraag,SCHEDULE,01-10-2011 00:39:38.875
4,4294967296,173691,2011-10-01T08:08:58.256+02:00,5000,A_SUBMITTED,COMPLETE,01-10-2011 08:08:58.256
...,...,...,...,...,...,...,...
214372,38835094290529,201854,2012-01-18T02:09:07.029+01:00,50000,O_CANCELLED,COMPLETE,14-03-2012 15:30:19.361
214373,38835094290528,201854,2012-01-18T02:09:07.029+01:00,50000,A_CANCELLED,COMPLETE,14-03-2012 15:30:19.361
214374,38835094290530,201854,2012-01-18T02:09:07.029+01:00,50000,W_Nabellen incomplete dossiers,COMPLETE,14-03-2012 15:30:23.187
214375,35858681954366,199678,2012-01-10T19:16:52.800+01:00,30000,W_Nabellen offertes,START,14-03-2012 15:36:15.299


In [8]:
df_test

,eventID,case concept:name,case REG_DATE,case AMOUNT_REQ,event concept:name,event lifecycle:transition,event time:timestamp
0,44964012621824,206324,2012-02-03T17:17:11.047+01:00,2500,A_SUBMITTED,COMPLETE,03-02-2012 17:17:11.047
1,44964012621825,206324,2012-02-03T17:17:11.047+01:00,2500,A_PARTLYSUBMITTED,COMPLETE,03-02-2012 17:17:11.323
2,44964012621826,206324,2012-02-03T17:17:11.047+01:00,2500,A_DECLINED,COMPLETE,03-02-2012 17:17:42.964
3,44968307589120,206327,2012-02-03T17:23:41.949+01:00,6000,A_SUBMITTED,COMPLETE,03-02-2012 17:23:41.949
4,44968307589121,206327,2012-02-03T17:23:41.949+01:00,6000,A_PARTLYSUBMITTED,COMPLETE,03-02-2012 17:23:42.504
...,...,...,...,...,...,...,...
47818,54666343743523,213276,2012-02-27T14:12:41.868+01:00,15000,W_Nabellen incomplete dossiers,START,14-03-2012 15:59:28.309
47819,54666343743524,213276,2012-02-27T14:12:41.868+01:00,15000,W_Nabellen incomplete dossiers,COMPLETE,14-03-2012 16:00:09.680
47820,49495203119136,209595,2012-02-15T10:10:36.503+01:00,13000,W_Nabellen offertes,START,14-03-2012 16:02:03.883
47821,52342766436386,211624,2012-02-21T23:38:40.044+01:00,35000,W_Nabellen incomplete dossiers,START,14-03-2012 16:04:46.192


In [9]:
df_train['event time:timestamp']= pd.to_datetime(df_train['event time:timestamp'])

df_train = df_train.sort_values(by=['case concept:name', 'event time:timestamp'])

df_train = df_train.assign(actualaction=(df_train.groupby('case concept:name')['event concept:name'].shift(-1)))

df_train = df_train.assign(nexttime=(df_train.groupby('case concept:name')['event time:timestamp'].shift(-1)))
df_train['tonext'] = (df_train.nexttime - df_train['event time:timestamp']).astype('timedelta64[s]')

In [ ]:
df_train